# LangChain Expression Language (LCEL)

The **L**ang**C**hain **E**xpression **L**anguage (LCEL) is a abstraction of some interesting Python concepts into a format that enables a "minimalist" code layer for building chains of LangChain components.

LCEL comes with strong support for:

1. Superfast development of chains.
2. Advanced features such as streaming, async, parallel execution, and more.
3. Easy integration with LangSmith and LangServe.

In [ ]:
#!pip install anthropic
#pip install cohere

In [ ]:
!pip install -qU \
    langchain==0.0.345 \
    anthropic==0.7.7 \
    cohere==4.37 \
    docarray==0.39.1

## LCEL Syntax

To understand LCEL syntax let's first build a simple chain in typical Python syntax.

In [183]:
from langchain.chat_models import ChatAnthropic
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser

In [185]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

ANTHROPIC_API_KEY= os.getenv('ANTHROPIC_API_KEY')
COHERE_API_KEY=os.getenv('COHERE_API_KEY')

In [187]:
prompt = ChatPromptTemplate.from_template(
    "Give me small report about {topic}"
)
model = ChatAnthropic(
    model="claude-2.1",
    max_tokens_to_sample=512,
    anthropic_api_key=ANTHROPIC_API_KEY
)  # swap Anthropic for OpenAI with `ChatOpenAI` and `openai_api_key`
output_parser = StrOutputParser()

In typical LangChain we would chain these together using an `LLMChain`:

In [190]:
from langchain.chains import LLMChain

chain = LLMChain(
    prompt=prompt,
    llm=model,
    output_parser=output_parser
)

# and run
out = chain.run(topic="Artificial Intelligence")
print(out)

 Here is a brief report on artificial intelligence:

Artificial intelligence (AI) refers to computer systems or machines that are designed to perform tasks that would otherwise require human intelligence such as visual perception, speech recognition, decision-making, and language translation. AI has seen major advances in recent years due to increased availability of data, advanced algorithms, and improvements in computing power through techniques like parallel processing and neural networks.

Some key highlights about the current state of AI:

- AI systems have matched or surpassed human performance in specialized tasks like playing complex games (e.g. chess, Go), image classification, and speech recognition. However, matching comprehensive human intelligence across all domains remains an aspirational goal for AI researchers.  

- Machine learning, which refers to algorithms that automatically improve themselves through experience and data without explicit programming, has been one of

Using LCEL the format is different, rather than relying on `Chains` we simple chain together each component using the pipe operator `|`:

In [192]:
lcel_chain = prompt | model | output_parser

# and run
out = lcel_chain.invoke({"topic": "Artificial Intelligence"})
print(out)

 Here is a brief report on artificial intelligence (AI):

Introduction 

- Artificial intelligence refers to computer systems that are designed to perform tasks that typically require human intelligence, such as visual perception, speech recognition, decision-making, and language translation.

- AI has advanced significantly in recent years due to increases in data availability, faster computers, and advances in machine learning techniques like deep learning.

Current Applications

- AI is currently used in many applications, such as:

- Chatbots and virtual assistants like Alexa, Siri - interact with humans using natural language processing.

- Image recognition software - identify and label objects in images and videos. Used on social media and self-driving cars.

- Recommender systems - provide personalized recommendations for content on Spotify and Netflix.

- Fraud detection - detect credit card and financial fraud by analyzing purchase data for anomalies. Banks save billions thro

Pretty cool, the way that this pipe operator is being used is that it is taking output from the function to the _left_ and feeding it into the function on the _right_.

## How the Pipe Operator Works

To really understand LCEL we can take a look at how this pipe operation works. We know it takes output from the _right_ and feeds it to the _left_ — but this isn't typical Python, so how is this implemented? Let's try creating our own version of this with some simple functions.

We will be using the `__or__` method within Python class objects. When we place two classes together like `chain = class_a | class_b` the Python interpreter will check if these classes contain this `__or__` method. If they do then our `|` code will be translated into `chain = class_a.__or__(class_b)`.

That means both of these patterns will return the same thing:

```python
# object approach
chain = class_a.__or__(class_b)
chain("some input")

# pipe approach
chain = class_a | class_b
chain("some input")

```

With that in mind, we can build a `Runnable` class that consumes a function and turns it into a function that can be chained with other functions using the pipe operator `|`.

In [196]:
class Runnable:
    def __init__(self, func):
        self.func = func

    def __or__(self, other):
        def chained_func(*args, **kwargs):
            # the other func consumes the result of this func
            return other(self.func(*args, **kwargs))
        return Runnable(chained_func)

    def __call__(self, *args, **kwargs):
        return self.func(*args, **kwargs)

Let's implement this to take the value `3`, add `5` (giving `8`), and multiply by `2` (giving `16`).

In [198]:
def add_five(x):
    return x + 5

def multiply_by_two(x):
    return x * 2

# wrap the functions with Runnable
add_five = Runnable(add_five)
multiply_by_two = Runnable(multiply_by_two)

# run them using the object approach
chain = add_five.__or__(multiply_by_two)
chain(3)  # should return 16

16

Using `__or__` directly we get the correct answer, now let's try using the pipe operator `|` to chain them together:

In [200]:
# chain the runnable functions together
chain = add_five | multiply_by_two

# invoke the chain
chain(3)  # we should return 16

16

Using either method we get the same response, at it's core this is what LCEL is doing, but there is more.

## LCEL Deep Dive

Now that we understand what this syntax is doing under the hood, let's explore it within the context of LCEL and see a few of the additional methods that LangChain has provided to maximize flexibility when working with LCEL.

### Runnables

When working with LCEL we may find that we need to modify the structure or values being passed between components — for this we can use _runnables_. Let's try:

In [206]:
from langchain.embeddings import CohereEmbeddings
from langchain.vectorstores import DocArrayInMemorySearch

embedding = CohereEmbeddings(
    model="embed-english-light-v3.0",
    cohere_api_key=COHERE_API_KEY
)

vecstore_a = DocArrayInMemorySearch.from_texts(
    ["half the info will be here", "James' birthday is the 7th December"],
    embedding=embedding
)
vecstore_b = DocArrayInMemorySearch.from_texts(
    ["and half here", "James was born in 1994"],
    embedding=embedding
)

First let's try passing a question through to a single one of these `vecstore` objects:

In [208]:
from langchain_core.runnables import (
    RunnableParallel,
    RunnablePassthrough
)

retriever_a = vecstore_a.as_retriever()
retriever_b = vecstore_b.as_retriever()

prompt_str = """Answer the question below using the context:

Context: {context}

Question: {question}

Answer: """
prompt = ChatPromptTemplate.from_template(prompt_str)

retrieval = RunnableParallel(
    {"context": retriever_a, "question": RunnablePassthrough()}
)

chain = retrieval | prompt | model | output_parser

We use two new objects here, `RunnableParallel` and `RunnablePassthrough`. The `RunnableParallel` object allows us to define multiple values and operations, and run them all in parallel. Here we call `retriever_a` using the input to our chain (below), and then pass the results from `retriever_a` to the next component in the `chain` via the `"context"` parameter.

<div>
<img src="./img/lecl.webp" alt='auto' width="1000"/>
</div>

The `RunnablePassthrough` object is used as a `"passthrough"` take takes any input to the current component (`retrieval`) and allows us to provide it in the component output via the `"question"` key.

In [ ]:
out = chain.invoke("when was James born?")
print(out)

Here we have used `RunnableParallel` to create two parallel streams of information, one for `"context"` that is information fed in from `retriever_a`, and another for `"question"` which is the _passthrough_ information, ie the information that is passed through from our `chain.invoke("when was James born?")` call.

Using this information the chain is close to answering the question but it doesn't have enough information, it is missing the information that we have stored in `retriever_b`. Fortunately, we can have multiple parallel information streams using the `RunnableParallel` object.

In [ ]:
prompt_str = """Answer the question below using the context:

Context:
{context_a}
{context_b}

Question: {question}

Answer: """
prompt = ChatPromptTemplate.from_template(prompt_str)

retrieval = RunnableParallel(
    {
        "context_a": retriever_a, "context_b": retriever_b,
        "question": RunnablePassthrough()
    }
)

chain = retrieval | prompt | model | output_parser

In [ ]:
out = chain.invoke("when was James born?")
print(out)

Now the `RunnableParallel` object is allowing us to search with `retriever_a` _and_ `retriever_b` in parallel, ie at the same time.

## Runnable Lambdas

The `RunnableLambda` is a LangChain abstraction that allows us to turn Python functions into pipe-compatible function, similar to the `Runnable` class we created near the beginning of this notebook.

Let's try it out with our earlier `add_five` and `multiply_by_two` functions.

In [ ]:
from langchain_core.runnables import RunnableLambda

# wrap the functions with RunnableLambda
add_five = RunnableLambda(add_five)
multiply_by_two = RunnableLambda(multiply_by_two)

As with our earlier `Runnable` abstraction, we can use `|` operators to chain together our `RunnableLambda` abstractions.

In [ ]:
chain = add_five | multiply_by_two

Unlike our `Runnable` abstraction, we cannot run the `RunnableLambda` chain by calling it directly, instead we must call `chain.invoke`:

In [ ]:
chain.invoke(3)

As before, we can see the same answer. Naturally we can feed custom functions into our chains using this approach. Let's try a short chain and see where we might want to insert a custom function:

In [ ]:
prompt_str = "Tell me an short fact about {topic}"
prompt = ChatPromptTemplate.from_template(prompt_str)

chain = prompt | model | output_parser

In [ ]:
chain.invoke({"topic": "Artificial Intelligence"})

In [ ]:
chain.invoke({"topic": "Artificial Intelligence"})

The returned text always includes the initial `"Here's a short fact about ...\n\n"` — let's add a function to split on double newlines `"\n\n"` and only return the fact itself.

In [ ]:
def extract_fact(x):
    if "\n\n" in x:
        return "\n".join(x.split("\n\n")[1:])
    else:
        return x
    
get_fact = RunnableLambda(extract_fact)

chain = prompt | model | output_parser | get_fact

In [ ]:
chain.invoke({"topic": "Artificial Intelligence"})

In [ ]:
chain.invoke({"topic": "Artificial Intelligence"})

Now we're getting well formatted outputs using our final custom `get_fact` function.

---

That covers the essentials you need to getting started and building with the **L**ang**C**hain **E**xpression **L**anguage (LCEL). With it, we can put together chains very easily — and the current focus of the LangChain team is on further LCEL development and support.

The pros and cons of LCEL are varied. Those that love it tend to focus on the minimalist code style, LCEL's support for streaming, parallel operations, and async, and LCEL's nice integration with LangChain's focus on chaining components together.

There are also people that are less fond of LCEL. Typically people will point to it being yet another abstraction on top of an already very abstract library, that the syntax is confusing, against [the Zen of Python](https://peps.python.org/pep-0020/), and requires too much effort to learn new (or uncommon) syntax.

Both viewpoints are entirely valid, LCEL is a very different approach — ofcourse there are major pros and major cons. In either case, if you're willing to spend some time learning the syntax, it allows us to develop very quickly, and with that in mind it's well worth learning.

---